# How to use few shot examples in chat models

This guide covers how to prompt a chat model with example inputs and outputs. Providing the model with a few such examples is called few-shotting, and is a simple yet powerful way to guide generation and in some cases drastically improve model performance.

There does not appear to be solid consensus on how best to do few-shot prompting, and the optimal prompt compilation will likely vary by model. Because of this, we provide few-shot prompt templates like the FewShotChatMessagePromptTemplate as a flexible starting point, and you can modify or replace them as you see fit.

The goal of few-shot prompt templates are to dynamically select examples based  on an input, and then format the examples in a final prompt to provide for the model.

**Note:** The following code examples are for chat models only, since `FewShotChatMessagePromptTemplates` are designed to output formatted chat message rather than pure strings. For similar few-shot prompt examples for pure string templates compatible with completion models(LLMs), see the few-shot prompt templates guide.

## Fixed Examples
The most basic(and common) few-shot prompting technique is to use fixed prompt examples. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production.

The basic components of the template are:
* `examples`: A list of dictionary examples to include the final prompt.
* `example_prompt`: converts each example into 1 or more messages through its `format_messages` method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.

Below is a simple demonstration. First, define the examples you'd like to include. Let's give the LLM an unfamiliar mathematical operator, denoted by the "$" :

In [1]:
import os, yaml
from langchain_community.chat_models.zhipuai import ChatZhipuAI

with open('utils/config.yml', 'r') as stream:
    zhipuai_api_key = yaml.safe_load(stream)['api_key']
os.environ['ZHIPUAI_API_KEY'] = zhipuai_api_key

model = ChatZhipuAI(
    model='glm-4-plus',
    temperature=0.0,
)

model.invoke('What is 2 $ 2?')


AIMessage(content='The expression "2 $ 2" is not a standard mathematical operation. The dollar sign ($) is typically used in programming languages and certain mathematical contexts, but its meaning can vary depending on the context. Here are a few possibilities:\n\n1. **Programming Languages**: In some programming languages, the dollar sign is used as a part of variable names (e.g., in PHP or JavaScript). However, "2 $ 2" would not be a valid expression in most languages.\n\n2. **Regular Expressions**: In regular expressions, the dollar sign is used to denote the end of a string. But "2 $ 2" would not make sense in this context either.\n\n3. **Custom Operators**: In some custom or domain-specific languages, the dollar sign might be defined as a specific operator. Without knowing the context, it\'s impossible to determine what "2 $ 2" would evaluate to.\n\n4. **Financial Context**: If this is in a financial context, it might represent some kind of monetary operation, but it\'s not stand

In [4]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {'input': '2 $ 2', 'output': '4'},
    {'input': '2 $ 3', 'output': '5'},
]

example_prompt = ChatPromptTemplate.from_messages(
    {
        ('human', '{input}'),
        ('ai','{output}')
    }
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='2 $ 2'), AIMessage(content='4'), HumanMessage(content='2 $ 3'), AIMessage(content='5')]


In [7]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system','you are a wondrous wizard of math.'),
        few_shot_prompt,
        ('human', '{input}')
    ]
)


In [10]:
chain = final_prompt | model

chain.invoke('what is 2 $ 9 ?')

AIMessage(content="It seems like you're using a custom operation denoted by the symbol `$`. Based on the previous examples:\n\n- \\( 2 $ 2 = 4 \\)\n- \\( 2 $ 3 = 5 \\)\n\nWe can infer a pattern. In both cases, the result is obtained by adding 2 to the second number:\n\n- \\( 2 $ 2 = 2 + 2 = 4 \\)\n- \\( 2 $ 3 = 2 + 3 = 5 \\)\n\nFollowing this pattern, for \\( 2 $ 9 \\):\n\n\\[ 2 $ 9 = 2 + 9 = 11 \\]\n\nSo, \\( 2 $ 9 \\) would be 11.", response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 42, 'total_tokens': 194}, 'model_name': 'glm-4-plus', 'finish_reason': 'stop'}, id='run-c7953465-e652-4102-a3e1-9f0618c6d843-0')